In [ ]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np
import scipy
from scipy.special import gamma
from scipy.special import eval_legendre

# Plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly

# Atoms
import ase
from ase.io import read
from rascal.representations import SphericalInvariants
from rascal.neighbourlist.structure_manager import AtomsList
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_species

# Utilities
import h5py
import json
from selection import FPS, random_selection
from project_utils import load_structures_from_hdf5

# SOAP
from soap import quippy_soap, librascal_soap

# SOAP sandbox

In [ ]:
def get_power_spectrum_real_space_features(hypers, frames):
    """
        Author: Alexander Goscinski
    """
    
    representation = SphericalInvariants(**hypers)
    
    # internal librascal list of atoms
    atom_list = AtomsList(frames, representation.nl_options)
    
    if hypers["normalize"]:
        print("Warning: To obtain real space features, normalization is set to False")
        hypers["normalize"] = False
    if hypers["radial_basis"] != "DVR":
        print("Warning: To obtain real space features, radial_basis is set to DVR")
        hypers["radial_basis"] = "DVR"
    
    # gives you idx map for each feature dimension
    # e.g. print(feature_idx_map[0])
    #     {'a': 6, 'b': 6, 'n1': 0, 'n2': 0, 'l': 0}
    # 'a' and 'b' are species in number form, n1 and n2 radial index, l angular index
    feature_idx_map = representation.get_feature_index_mapping(atom_list)
    
    # Gaussian quadrature point weights, sqrt(weights) 
    radial_spectrum_weights = np.sqrt(np.polynomial.legendre.leggauss(hypers["max_radial"])[1])
    
    # list broadcasting is faster than for loops
    power_spectrum_weights = np.array([radial_spectrum_weights[idx['n1']]*radial_spectrum_weights[idx['n2']] 
                                       for _, idx in feature_idx_map.items()])
    
    features = representation.transform(atom_list).get_features(representation)
    
    return features / power_spectrum_weights[np.newaxis, :]

In [ ]:
# Load DEEM 10k
deem_10k = read('../Raw_Data/DEEM_10k/DEEM_10000.xyz', index='0:10')

In [ ]:
# Setup SOAP parameters
soap_hyperparameters = dict(max_radial=12,
                            max_angular=9,
                            cutoff_smooth_width=0.3,
                            gaussian_sigma_constant=0.3)

In [ ]:
soaps_gto = librascal_soap(deem_10k, [14],
                           interaction_cutoff=6.0,
                           radial_basis='GTO',
                           **soap_hyperparameters)

In [ ]:
soaps_dvr = librascal_soap(deem_10k, [14],
                           interaction_cutoff=6.0,
                           radial_basis='DVR',
                           **soap_hyperparameters)

In [ ]:
soaps_gto[0]

In [ ]:
soaps_dvr[0]

In [ ]:
for frame in deem_10k:
    frame.center()
    frame.wrap(eps=1.0E-12)

In [ ]:
# Do GTO computation manually
gto_representation = SphericalInvariants(interaction_cutoff=6.0,
                                         gaussian_sigma_type='Constant',
                                         radial_basis='GTO',
                                         **soap_hyperparameters)
deem_gto_rep = gto_representation.transform(deem_10k)
deem_gto_features = deem_gto_rep.get_features(gto_representation)

In [ ]:
deem_gto_features[0:8]

In [ ]:
# Do DVR computation manually
dvr_representation = SphericalInvariants(interaction_cutoff=6.0,
                                         gaussian_sigma_type='Constant',
                                         radial_basis='DVR',
                                         **soap_hyperparameters)
deem_dvr_rep = dvr_representation.transform(deem_10k)
deem_dvr_features = deem_dvr_rep.get_features(dvr_representation)

In [ ]:
deem_dvr_features[0:8]

In [ ]:
# gives you idx map for each feature dimension
# e.g. print(feature_idx_map[0])
#     {'a': 6, 'b': 6, 'n1': 0, 'n2': 0, 'l': 0}
# 'a' and 'b' are species in number form, n1 and n2 radial index, l angular index
feature_idx_map = dvr_representation.get_feature_index_mapping(deem_10k)

# Gaussian quadrature point weights, sqrt(weights) 
radial_spectrum_weights = np.sqrt(np.polynomial.legendre.leggauss(soap_hyperparameters["max_radial"])[1])

# list broadcasting is faster than for loops
power_spectrum_weights = np.array([radial_spectrum_weights[idx['n1']]*radial_spectrum_weights[idx['n2']] 
                                   for _, idx in feature_idx_map.items()])

In [ ]:
deem_dvr_features /= power_spectrum_weights

In [ ]:
feature_idx_map

In [ ]:
# Number of components: (n_species)*(n_species + 1) / 2 * n_max ** 2 * (l_max + 1)
soaps_gto[0].shape

# Simple structure

In [ ]:
frames = [ase.Atoms('CC'), ase.Atoms('CCC')]
frames[0][1].position = [0,2,0]
frames[1][1].position = [0,2,0]
frames[1][2].position= [1,1,0]
for i in range(len(frames)):
    frames[i].cell = np.eye(3) * 15
    frames[i].center()

In [ ]:
frames[0].positions

In [ ]:
a = 3
frames = [ase.Atoms('Si', positions=[[0, 0, 0]],
                    cell=[[a, 0, 0],
                          [0, a, 0],
                          [0, 0, a]],
                    pbc=[1, 1, 1])]

# GTO Radial spectrum

In [ ]:
def rascal_basis(prefactor, b, n, r):
    '''
    Rascal uses T_n(r) = r R_n(r) as basis function for the radial part of the
    expansion, where R_n(r) is a GTO function: R_n(r) = r^n exp(-r^2 / 2 *
    sigma^2). This function compute one of such basis function.
    '''
    return prefactor * r ** (n+1) * np.exp(-b * r ** 2)


def overlap_matrix(prefactor, b, n):
    '''
    Compute the overlap matrix between Tn(r) function to then orthonormalized
    them.
    '''
    n_dash = n[:, np.newaxis]
    b_dash = b[:, np.newaxis]
    prefactor_dash = prefactor[:, np.newaxis]

    n_n_dash = n + n_dash
    b_b_dash = b + b_dash

    overlap = b_b_dash**(-0.5 * (3 + n_n_dash)) * gamma(0.5 * (3 + n_n_dash))
    overlap = 0.5 * prefactor * prefactor_dash * overlap
    
    return overlap


def normalized_radial_basis(max_radial, cutoff):
    '''
    Get a function r -> [^T_0(r), ^T_1(r), ..., ^T_n_max(r)], where ^T_n(r) is
    the orthonormalized version of T_n(r)

    In other words, get the basis functions used when decomposing the density in
    rascal.
    '''
    n = np.arange(max_radial)
    sigma = np.sqrt(np.hstack((1, np.arange(1, max_radial)))) * cutoff / max_radial
    prefactor = np.sqrt(2 / (sigma ** (2 * n + 3) * gamma(n + 3 / 2)))
    b = 1 / (2 * sigma ** 2)
    overlap = overlap_matrix(prefactor, b, n)
    S_half = scipy.linalg.fractional_matrix_power(overlap, -0.5)
    print(S_half)
    
    return lambda r: S_half.dot(rascal_basis(prefactor, b, n, r))


def density_from_soap(soap_vector, nb_species, max_radial, cutoff, grid):
    '''
    Get the spherically averaged density evaluated on the given grid
    '''
    basis_functions = normalized_radial_basis(max_radial, cutoff)
    print(basis_functions(grid[0]).shape)
    basis = np.vstack([basis_functions(r) for r in grid]).T
    print(basis.shape)
    dr = np.diff(grid)[0]
    for n in range(0, max_radial):
        for m in range(0, max_radial):
            print(n, m)
            print(np.sum(basis[n] * basis[m]) * dr)
            
    
    return np.array([soap_vector.dot(np.tile(basis_functions(r), nb_species)) for r in grid]).T

hypers_rs = {
    "soap_type": "RadialSpectrum",
    "radial_basis": "GTO",
    "interaction_cutoff": 4,
    "max_radial": 4,
    "max_angular": 0,
    "gaussian_sigma_constant": 0.5,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "normalize": False
}

representation_rs = SphericalInvariants(**hypers_rs)
features_rs = representation_rs.transform(frames).get_features(representation_rs)
grid = np.linspace(0.01,4,100)
plt.plot(grid, density_from_soap(features_rs, 1, hypers_rs["max_radial"], hypers_rs["interaction_cutoff"], grid).T)

# Refined GTO radial spectrum 

In [ ]:
def gto_basis_function(prefactor, b, n, r):
    '''
    Rascal uses T_n(r) = r R_n(r) as basis function for the radial part of the
    expansion, where R_n(r) is a GTO function: R_n(r) = r^n exp(-r^2 / 2 *
    sigma^2). This function compute one of such basis function.
    '''
    return prefactor * r ** (n+1) * np.exp(-b * r ** 2)

def compute_rs_r_grid(max_radial, cutoff, r_grid):
    n = np.arange(max_radial)
    sigma = np.sqrt(np.hstack((1, np.arange(1, max_radial)))) * cutoff / max_radial
    prefactor = np.sqrt(2 / (sigma ** (2 * n + 3) * gamma(n + 3 / 2)))
    b = 1 / (2 * sigma ** 2)
    
    return gto_basis_function(prefactor[:,np.newaxis], b[:,np.newaxis], n[:,np.newaxis], r_grid[np.newaxis, :])


def overlap_matrix(prefactor, b, n):
    '''
    Compute the overlap matrix between Tn(r) function to then orthonormalized
    them.
    '''
    n_dash = n[:, np.newaxis]
    b_dash = b[:, np.newaxis]
    prefactor_dash = prefactor[:, np.newaxis]

    n_n_dash = n + n_dash
    b_b_dash = b + b_dash

    overlap = b_b_dash**(-0.5 * (3 + n_n_dash)) * gamma(0.5 * (3 + n_n_dash))
    overlap = 0.5 * prefactor * prefactor_dash * overlap
    return overlap


def compute_S_half(max_radial, cutoff):
    '''
    Get a function r -> [^T_0(r), ^T_1(r), ..., ^T_n_max(r)], where ^T_n(r) is
    the orthonormalized version of T_n(r)

    In other words, get the basis functions used when decomposing the density in
    rascal.
    '''
    n = np.arange(max_radial)
    sigma = np.sqrt(np.hstack((1, np.arange(1, max_radial)))) * cutoff / max_radial
    prefactor = np.sqrt(2 / (sigma ** (2 * n + 3) * gamma(n + 3 / 2)))
    b = 1 / (2 * sigma ** 2)
    overlap = overlap_matrix(prefactor, b, n)
    
    return scipy.linalg.fractional_matrix_power(overlap, -0.5)


def density_from_soap(normalized_radial_gto_rs_coeffs, nb_species, max_radial, cutoff, r_grid):
    '''
    Get the spherically averaged density evaluated on the given grid
    '''
    
    # The basis normalization matrix
    S_half = compute_S_half(max_radial, cutoff)
    
    # undo normalization
    radial_gto_rs_coeffs = np.zeros(normalized_radial_gto_rs_coeffs.shape)
    
    for i in range(nb_species):
        radial_gto_rs_coeffs[:,i:(i+1)*max_radial] = normalized_radial_gto_rs_coeffs[:,i:(i+1)*max_radial].dot(S_half)
    return radial_gto_rs_coeffs.dot(compute_rs_r_grid(max_radial, cutoff, r_grid))

hypers_rs = {
    "soap_type": "RadialSpectrum",
    "radial_basis": "GTO",
    "interaction_cutoff": 4,
    "max_radial": 20,
    "max_angular": 0,
    "gaussian_sigma_constant": 0.5,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "normalize": False
}


representation_rs = SphericalInvariants(**hypers_rs)
features_rs = representation_rs.transform(frames).get_features(representation_rs)
grid = np.linspace(0.01,4,100)
plt.plot(grid, density_from_soap(features_rs, 1, hypers_rs["max_radial"], hypers_rs["interaction_cutoff"], grid).T)

# GTO power spectrum

In [ ]:
def gto_basis_function(prefactor, b, n, r):
    '''
    Rascal uses T_n(r) = r R_n(r) as basis function for the radial part of the
    expansion, where R_n(r) is a GTO function: R_n(r) = r^n exp(-r^2 / 2 *
    sigma^2). This function compute one of such basis function.
    '''
    return prefactor * r ** (n+1) * np.exp(-b * r ** 2)


def overlap_matrix(prefactor, b, n):
    '''
    Compute the overlap matrix between Tn(r) function to then orthonormalized
    them.
    '''
    n_dash = n[:, np.newaxis]
    b_dash = b[:, np.newaxis]
    prefactor_dash = prefactor[:, np.newaxis]

    n_n_dash = n + n_dash
    b_b_dash = b + b_dash

    overlap = b_b_dash**(-0.5 * (3 + n_n_dash)) * gamma(0.5 * (3 + n_n_dash))
    overlap = 0.5 * prefactor * prefactor_dash * overlap
    print(overlap)
    return overlap


def compute_S_half(max_radial, cutoff):
    '''
    Get a function r -> [^T_0(r), ^T_1(r), ..., ^T_n_max(r)], where ^T_n(r) is
    the orthonormalized version of T_n(r)

    In other words, get the basis functions used when decomposing the density in
    rascal.
    '''
    n = np.arange(max_radial)
    sigma = np.sqrt(np.hstack((1, np.arange(1, max_radial)))) * cutoff / max_radial
    prefactor = np.sqrt(2 / (sigma ** (2 * n + 3) * gamma(n + 3 / 2)))
    b = 1 / (2 * sigma ** 2)
    overlap = overlap_matrix(prefactor, b, n)
    return scipy.linalg.fractional_matrix_power(overlap, -0.5)


def compute_ps_r_grid(max_radial, cutoff, radial_gto_rs_coeffs, r_grid, n_idx):
    n = np.arange(max_radial)
    sigma = np.sqrt(np.hstack((1, np.arange(1, max_radial)))) * cutoff / max_radial
    prefactor = np.sqrt(2 / (sigma ** (2 * n + 3) * gamma(n + 3 / 2)))
    b = 1 / (2 * sigma ** 2)
    
    # returns (nb_samples, nb_features, nb_grid_points) shape
    return ( radial_gto_rs_coeffs[:,n_idx][:,:, np.newaxis] * 
        gto_basis_function(prefactor[n_idx][:,np.newaxis], b[n_idx][:,np.newaxis],
                           n[n_idx][:,np.newaxis], r_grid[np.newaxis,:])[np.newaxis,:,:] )
    

def compute_ps_theta_grid(theta_grid, l_idx):
    
    # Tile inputs to evaluate the Legendre polynomials in one call
    tiled_theta_grid = np.tile(theta_grid, len(l_idx))
    tiled_l_idx = np.repeat(l_idx, len(theta_grid))
    
    legendre_evals = eval_legendre(tiled_l_idx, tiled_theta_grid)
    #legendre_evals = eval_legendre(np.amax(l_idx).repeat(len(tiled_theta_grid)), tiled_theta_grid)
    
    # returns (nb_features, nb_grid_points) shape
    return np.reshape(legendre_evals, (len(l_idx), len(theta_grid)))
    #return np.ones((len(l_idx), len(theta_grid)))


# we use same grid for r_grid and r'_grid
def compute_density(hypers, frames, r_grid, theta_grid):
    hypers = hypers.copy()
    if hypers["normalize"]:
        print("Warning: normalize was set to True. Solution will return distorted results")
    max_radial = hypers["max_radial"]
    cutoff = hypers["interaction_cutoff"]
    
    # get power spectrum feature idx
    representation = SphericalInvariants(**hypers)
    atom_list = AtomsList(frames, representation.nl_options)
    soap_coeffs = representation.transform(frames).get_features(representation)
    feature_idx_map = representation.get_feature_index_mapping(atom_list)
    n_idx = [idx['n1'] for _, idx in feature_idx_map.items()]
    nd_idx = [idx['n2'] for _, idx in feature_idx_map.items()]
    l_idx = [idx['l'] for _, idx in feature_idx_map.items()]
    
    # get unnormalized radial gto coeffs
    hypers["soap_type"] = "RadialSpectrum"
    hypers["max_angular"] = 0
    representation = SphericalInvariants(**hypers)
    normalized_radial_gto_rs_coeffs = representation.transform(frames).get_features(representation)
    
    # The basis normalization matrix
    S_half = compute_S_half(max_radial, cutoff)
    
    # undo normalization
    radial_gto_rs_coeffs = np.zeros(normalized_radial_gto_rs_coeffs.shape)
    nb_species = 1
    
    for i in range(nb_species):
        radial_gto_rs_coeffs[:,i:(i+1)*max_radial] = normalized_radial_gto_rs_coeffs[:,i:(i+1)*max_radial].dot(S_half)
           
    # c_n R_n(r) with (nb_samples, nb_features, nb_grid_points) shape
    radial_contribution_r = compute_ps_r_grid(max_radial, cutoff, radial_gto_rs_coeffs, r_grid, n_idx)
    
    # c_n' R_n'(r') with (nb_samples, nb_features, nb_grid_points) shape
    radial_contribution_rd = compute_ps_r_grid(max_radial, cutoff, radial_gto_rs_coeffs, r_grid, nd_idx)
    
    # c_n R_n(r) * c_n' R_n'(r') with (nb_samples, nb_features, nb_grid_points, nb_grid_points) shape
    radial_contribution = radial_contribution_r[:, :, :, np.newaxis] * radial_contribution_rd[:, :, np.newaxis, :]
    
    plt.imshow(radial_contribution.sum(axis=1)[0])
    plt.show()
    
    # extract angular coeffs
    soap_radial_coeffs = normalized_radial_gto_rs_coeffs[:,n_idx] * normalized_radial_gto_rs_coeffs[:,nd_idx]
    #soap_radial_coeffs = radial_gto_rs_coeffs[:,n_idx] * radial_gto_rs_coeffs[:,nd_idx]
    soap_angular_coeffs = soap_coeffs/soap_radial_coeffs
    #soap_angular_coeffs[soap_angular_coeffs > 1.0] = 10.0
    tmp = soap_radial_coeffs*soap_angular_coeffs
    print(np.allclose(tmp, soap_coeffs, atol=1.0E-12, rtol=1.0E-12))
    print(np.allclose(soap_coeffs, tmp, atol=1.0E-12, rtol=1.0E-12))
    
    # Looks like some divide by zero is artificially inflating the angular bits,
    # which inflate the zero radial parts?
    plt.imshow(soap_coeffs, aspect=100)
    plt.show()
    plt.imshow(soap_radial_coeffs, aspect=100)
    plt.show()
    plt.imshow(soap_angular_coeffs, aspect=100)
    plt.show()
    
    plt.plot(soap_coeffs[0][:])
    plt.title('soap_coeffs')
    plt.show()
    plt.plot(soap_radial_coeffs[0][:])
    plt.title('radial_coeffs')
    plt.show()
    plt.plot(soap_angular_coeffs[0][:]) # <-- issue?
    plt.title('angular_coeffs = soap_coeffs/radial_coeffs')
    plt.show()
        
    # computes legendre points for grid with shape (nb_features, nb_l_grid_points)
    legendre_contribution = compute_ps_theta_grid(theta_grid, l_idx)
    #legendre_contribution = compute_ps_theta_grid(theta_grid, l_idx)/(np.amax(l_idx)+1)

    legendre_sum = np.zeros(len(theta_grid))
    for l in range(0, l_max+1):
        legendre_sum += eval_legendre(l, theta_grid)
    
    plt.plot(legendre_sum*len(theta_grid))
    plt.plot(legendre_contribution.sum(axis=0))
    plt.plot(eval_legendre(np.amax(l_idx), theta_grid)*len(theta_grid))
    plt.show()
    
    angular_contribution = soap_angular_coeffs[:,:,np.newaxis] * legendre_contribution[np.newaxis,:,:] 
    #angular_contribution = legendre_contribution[np.newaxis,:,:]
    
    plt.plot(angular_contribution.sum(axis=1)[0])
    plt.show()
    
    # returns with shape (nb_samples, nb_features, nb_r_grid_points, nb_l_grid_points) matrix for legendre polynomials
    return radial_contribution[:,:,:,:, np.newaxis] * angular_contribution[:,:,np.newaxis,np.newaxis,:]
    
l_max = 4
hypers = {
    "soap_type": "PowerSpectrum",
    "radial_basis": "GTO",
    "interaction_cutoff": 6,
    "max_radial": 10,
    "max_angular": l_max,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "normalize": False
}
r_grid = np.linspace(0,4,50)
theta_grid = np.linspace(0,np.pi,100)
theta_grid = np.cos(theta_grid)
theta_grid = np.linspace(-1,1,100)

#shape is (nb_samples, nb_features, nb_r_grid_points, nb_r_grid_points, nb_theta_grid_points)
density_grid = compute_density(hypers, frames[0], r_grid, theta_grid)
print(density_grid.shape)

# to obain density for each sample
density = np.sum(density_grid, axis=1)
print(density.shape)
# shape now (nb_samples, nb_r_grid_points, nb_r_grid_points, nb_theta_grid_points)
# some 3d plot ... 
# because there is no angular information at the moment,
# we can take a (r_grid, r'_grid) slice for the first environment
plt.figure(figsize=(10,10))
plt.imshow(density[0,:,:,99])
plt.colorbar()
plt.show()

In [ ]:
legendre_sum = np.zeros(len(theta_grid))
for l in range(0, l_max+1):
    legendre_sum += eval_legendre(l, theta_grid)
    
plt.plot(legendre_sum*100)

In [ ]:
X, Y, Z = np.meshgrid(r_grid, r_grid, theta_grid)

In [ ]:
fig = go.Figure(data=go.Volume(x=X.flatten(),
                               y=Y.flatten(),
                               z=Z.flatten(),
                               value=density[0].flatten(),
                               isomin=5.0E-5,
                               isomax=None,#0.5*density[0].max(),
                               opacity=0.2,
                               surface_count=20))
fig.show()
#fig.write_html('CC-n10-l4-c4-cw0.5-gs0.3.html')